In [15]:

import cantera as ct
import numpy as np

#Parameters
T=383 #Initial temperature in K
p=1 #pressure in bar
volume = 0.1e-6 #volume of the catalyst zone
beta=20.0 #temperature ramp in K/min
m_cat=0.0413 #catalyst mass in g
cat_area = 10.4 #area in m2/g 
facet_fraction=[0.6923, 0.0439, 0.211, 0.0528] # [111,211, 100, 110]
sccm = 30.652*1e-6/60 #Flow rate m3/s

# input file containing the surface reaction mechanism
cti_file = 'original_mechanism_JPCC_2021/Multifacet_mechanism.yaml'

# import the gas model and set the initial conditions
gas = ct.Solution(cti_file, 'gas')
gas.TPX = T, p*ct.one_atm, 'Ar:1, He:0.0, CO2:0.0'

#import the (111) surface
surf111 = ct.Interface(cti_file,'surface1', [gas])
surf111.TP = T, p*ct.one_atm
surf111.coverages = {'site(111)':1 , 'O(111)':0.0, 'OC(111)':0.0 , 'CO2(111)':0}

#import the (211) surface
surf211 = ct.Interface(cti_file,'surface2', [gas])
surf211.TP = T, p*ct.one_atm
surf211.coverages = {'site(211)':1,  'O(211)':0.0, 'OC(211)':0.0 , 'CO2(211)':0}

#import the (100) surface
surf100 = ct.Interface(cti_file,'surface3', [gas])
surf100.TP = T, p*ct.one_atm
surf100.coverages = {'site(100)':1, 'O(100)':0.0, 'OC(100)':0.0, 'CO2(100)':0}

#import the (110) surface
surf110 = ct.Interface(cti_file,'surface4', [gas])
surf110.TP = T, p*ct.one_atm
surf110.coverages = {'site(110)':1, 'O(110)':0.0 , 'OC(110)':0.0  , 'CO2(110)':0}

#typical Cantera setup
upstream = ct.Reservoir(gas, name='upstream')
downstream = ct.Reservoir(gas, name='downstream')

r=ct.IdealGasReactor(gas, energy='off')
r.volume=volume
rsurf111=ct.ReactorSurface(surf111, r, A=facet_fraction[0]*cat_area*m_cat)
rsurf211=ct.ReactorSurface(surf211, r, A=facet_fraction[1]*cat_area*m_cat)
rsurf100=ct.ReactorSurface(surf100, r, A=facet_fraction[2]*cat_area*m_cat)
rsurf110=ct.ReactorSurface(surf110, r, A=facet_fraction[3]*cat_area*m_cat)

#This is important since it adjusts the flow rates, 303 K is the temperature of the gas cylinders
vdot = sccm  * ((ct.one_atm / gas.P) * ( 303 / 273.15)) # m^3/s 
mass_flow_rate =  vdot* gas.density
mflow = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)
v = ct.PressureController(r, downstream, master=mflow, K=1e-9)

sim=ct.ReactorNet([r])

##set relative and absolute tolerances on the simulation
sim.rtol = 1.0e-12
sim.atol = 1.0e-22
sim.max_err_test_fails = 10000
rxn_time=np.linspace(0,8400,8400)

#get indices of the gas and surface species, (211) and (111) have the same order of species
i_co2=gas.species_index('CO2')
i_co=gas.species_index('CO')

i_surf_Ni=surf111.species_index('site(111)')
i_surf_CO=surf111.species_index('OC(111)')
i_surf_O=surf111.species_index('O(111)')
i_surf_C=surf111.species_index('C(111)')
i_surf_CO2=surf111.species_index('CO2(111)')

gas_mole_fracs=np.zeros([gas.n_species, len(rxn_time)])

Ti=np.zeros(len(rxn_time))
sim.max_time_step=1e-4
#Run the simulation
for i in range(len(rxn_time)):
        time=rxn_time[i]
        #print(time)
        if time < 10:
            upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0.0, CO2:0.0'
            upstream.syncState()
        elif (time > 10) and (time < 3610):   
            sim.max_time_step=1
            upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:0, He:0.9013, CO2:0.0987'
            upstream.syncState()
        elif (time > 3610) and (time < 3970): 
            sim.max_time_step=10
            upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0, CO2:0'
            upstream.syncState()
        elif (time > 3970) and (time < 5410):
            sim.max_time_step=10
            upstream.thermo.TPX=323,p*ct.one_atm, 'Ar:1, He:0, CO2:0'   
            upstream.syncState()
            r.thermo.TP=323,ct.one_atm
            r.syncState()
        else:
            r.thermo.TP=323+beta/60.0*(rxn_time[i]-5410),ct.one_atm
            r.syncState()
             
        sim.reinitialize  
        sim.advance(time)
        Ti[i]=r.thermo.T
        gas_mole_fracs[:,i]=gas.X
        
#Get the maximum and the position of the maximum of the TPD profile
ind=5410

ref_height_CO2=np.max(gas_mole_fracs[i_co2, ind:])*1e6
ref_Temp_CO2=Ti[np.argmax(gas_mole_fracs[i_co2,ind:])+5410]

ref_height_CO=np.max(gas_mole_fracs[i_co, ind:])*1e6
ref_Temp_CO=Ti[np.argmax(gas_mole_fracs[i_co,ind:])+5410]

print(ref_height_CO2,ref_Temp_CO2,ref_height_CO,ref_Temp_CO)

547.8440940244743 420.2262570940987 182.3048386021555 651.9205064094933


In [16]:
##run the sensitivity analysis for the 111 facet

heights_CO2_111,heights_CO_111=[],[]
temps_CO2_111,temps_CO_111=[],[]
  
dH=1.0e-0
dk = dH*1000 / 8.314  # for the thermo loop, 'dk' is in fact (delta H / R)
for count,j in enumerate(surf111.species_names[1:]):    
    # import the gas model and set the initial conditions
    gas = ct.Solution(cti_file, 'gas')
    gas.TPX = T, p*ct.one_atm, 'Ar:1, He:0.0, CO2:0.0'

    #import the (111) surface
    surf111 = ct.Interface(cti_file,'surface1', [gas])
    surf111.TP = T, p*ct.one_atm
    surf111.coverages = {'site(111)':1 , 'O(111)':0.0, 'OC(111)':0.0 , 'CO2(111)':0}

    #import the (211) surface
    surf211 = ct.Interface(cti_file,'surface2', [gas])
    surf211.TP = T, p*ct.one_atm
    surf211.coverages = {'site(211)':1,  'O(211)':0.0, 'OC(211)':0.0 , 'CO2(211)':0}

    #import the (100) surface
    surf100 = ct.Interface(cti_file,'surface3', [gas])
    surf100.TP = T, p*ct.one_atm
    surf100.coverages = {'site(100)':1, 'O(100)':0.0, 'OC(100)':0.0, 'CO2(100)':0}

    #import the (110) surface
    surf110 = ct.Interface(cti_file,'surface4', [gas])
    surf110.TP = T, p*ct.one_atm
    surf110.coverages = {'site(110)':1, 'O(110)':0.0 , 'OC(110)':0.0  , 'CO2(110)':0}

    #typical Cantera setup
    upstream = ct.Reservoir(gas, name='upstream')
    downstream = ct.Reservoir(gas, name='downstream')

    r=ct.IdealGasReactor(gas, energy='off')
    r.volume=volume
    rsurf111=ct.ReactorSurface(surf111, r, A=facet_fraction[0]*cat_area*m_cat)
    rsurf211=ct.ReactorSurface(surf211, r, A=facet_fraction[1]*cat_area*m_cat)
    rsurf100=ct.ReactorSurface(surf100, r, A=facet_fraction[2]*cat_area*m_cat)
    rsurf110=ct.ReactorSurface(surf110, r, A=facet_fraction[3]*cat_area*m_cat)

    #This is important since it adjusts the flow rates, 303 K is the temperature of the gas cylinders
    vdot = sccm  * ((ct.one_atm / gas.P) * ( 303 / 273.15)) # m^3/s 
    mass_flow_rate =  vdot* gas.density
    mflow = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)
    v = ct.PressureController(r, downstream, master=mflow, K=1e-9)

    sim=ct.ReactorNet([r])

    ##set relative and absolute tolerances on the simulation
    sim.rtol = 1.0e-12
    sim.atol = 1.0e-22
    sim.max_err_test_fails = 10000
    rxn_time=np.linspace(0,8400,8400)

    #get indices of the gas and surface species, (211) and (111) have the same order of species
    i_co2=gas.species_index('CO2')
    i_co=gas.species_index('CO')

    i_surf_Ni=surf111.species_index('site(111)')
    i_surf_CO=surf111.species_index('OC(111)')
    i_surf_O=surf111.species_index('O(111)')
    i_surf_C=surf111.species_index('C(111)')
    i_surf_CO2=surf111.species_index('CO2(111)')

    gas_mole_fracs_111=np.zeros([gas.n_species, len(rxn_time)])
    

    #get the index for each species
    idx=surf111.species_index(j)
    #get the species 
    s=surf111.species(idx)
    print(s)
    #call the original coefficients of the NASA polynomial
    original_coeffs = s.thermo.coeffs
    #create an empty array for the perturbation
    perturbed_coeffs = np.ones_like(original_coeffs)
    #fill the array initially with the original coeffs
    perturbed_coeffs[:] = original_coeffs
    #perturb the enthalpy of the adsorbate for the low and high temperature polynomial
    perturbed_coeffs[6] = original_coeffs[6] - dk
    #This is not a mistake, both values have to be perturbed by exactly the same value, otherwise discontinuity
    perturbed_coeffs[13] = original_coeffs[13] - dk
    #reset all multipliers
            
    #fit a new NASA polynomial through the data and update the thermo database
    s.thermo = ct.NasaPoly2(100.000, 5000.000, ct.one_atm, perturbed_coeffs)
    surf111.modify_species(idx, s) 

    Ti=np.zeros(len(rxn_time))
    sim.max_time_step=1e-4
    #Run the simulation
    for i in range(len(rxn_time)):
            time=rxn_time[i]
            #print(time)
            if time < 10:
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0.0, CO2:0.0'
                upstream.syncState()
            elif (time > 10) and (time < 3610):   
                sim.max_time_step=1
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:0, He:0.9013, CO2:0.0987'
                upstream.syncState()
            elif (time > 3610) and (time < 3970): 
                sim.max_time_step=10
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0, CO2:0'
                upstream.syncState()
            elif (time > 3970) and (time < 5410):
                sim.max_time_step=10
                upstream.thermo.TPX=323,p*ct.one_atm, 'Ar:1, He:0, CO2:0'   
                upstream.syncState()
                r.thermo.TP=323,ct.one_atm
                r.syncState()
            else:
                r.thermo.TP=323+beta/60.0*(rxn_time[i]-5410),ct.one_atm
                r.syncState()
             
            sim.reinitialize  
            sim.advance(time)
            Ti[i]=r.thermo.T
            gas_mole_fracs_111[:,i]=gas.X

    ind=5410
    
    max_height_CO2=np.max(gas_mole_fracs_111[i_co2,ind:])*1e6
    max_Temp_CO2=Ti[np.argmax(gas_mole_fracs_111[i_co2,ind:])+ind] 
    
    max_height_CO=np.max(gas_mole_fracs_111[i_co,ind:])*1e6
    max_Temp_CO=Ti[np.argmax(gas_mole_fracs_111[i_co,ind:])+ind]  
    print(max_height_CO2,max_Temp_CO2,max_height_CO,max_Temp_CO)
    heights_CO2_111.append(max_height_CO2)
    temps_CO2_111.append(max_Temp_CO2)

    heights_CO_111.append(max_height_CO)
    temps_CO_111.append(max_Temp_CO)
    
    #reset all multiplier
    #restore the original NASA polynomial and perturb the next species (only one at a time)
    s.thermo = ct.NasaPoly2(100.000, 5000.000, ct.one_atm, original_coeffs)
    surf111.modify_species(idx, s) 


<Species O(111)>
637.9413281877831 423.2266142794779 182.29599415318498 651.9205064094933
<Species OC(111)>
630.3390635538616 423.2266142794779 184.49938732809986 655.9209826566654
<Species C(111)>
547.8440940244743 420.2262570940987 182.30483859675394 651.9205064094933
<Species CO2(111)>
504.2830015554952 417.2258999087192 182.31571326497206 651.9205064094933


In [17]:
##run the sensitivity analysis for the 211 facet

heights_CO2_211,heights_CO_211=[],[]
temps_CO2_211,temps_CO_211=[],[]
  
dH=1.0e-0
dk = dH*1000 / 8.314  # for the thermo loop, 'dk' is in fact (delta H / R)
for count,j in enumerate(surf211.species_names[1:]):    
    # import the gas model and set the initial conditions
    gas = ct.Solution(cti_file, 'gas')
    gas.TPX = T, p*ct.one_atm, 'Ar:1, He:0.0, CO2:0.0'

    #import the (111) surface
    surf111 = ct.Interface(cti_file,'surface1', [gas])
    surf111.TP = T, p*ct.one_atm
    surf111.coverages = {'site(111)':1 , 'O(111)':0.0, 'OC(111)':0.0 , 'CO2(111)':0}

    #import the (211) surface
    surf211 = ct.Interface(cti_file,'surface2', [gas])
    surf211.TP = T, p*ct.one_atm
    surf211.coverages = {'site(211)':1,  'O(211)':0.0, 'OC(211)':0.0 , 'CO2(211)':0}

    #import the (100) surface
    surf100 = ct.Interface(cti_file,'surface3', [gas])
    surf100.TP = T, p*ct.one_atm
    surf100.coverages = {'site(100)':1, 'O(100)':0.0, 'OC(100)':0.0, 'CO2(100)':0}

    #import the (110) surface
    surf110 = ct.Interface(cti_file,'surface4', [gas])
    surf110.TP = T, p*ct.one_atm
    surf110.coverages = {'site(110)':1, 'O(110)':0.0 , 'OC(110)':0.0  , 'CO2(110)':0}

    #typical Cantera setup
    upstream = ct.Reservoir(gas, name='upstream')
    downstream = ct.Reservoir(gas, name='downstream')

    r=ct.IdealGasReactor(gas, energy='off')
    r.volume=volume
    rsurf111=ct.ReactorSurface(surf111, r, A=facet_fraction[0]*cat_area*m_cat)
    rsurf211=ct.ReactorSurface(surf211, r, A=facet_fraction[1]*cat_area*m_cat)
    rsurf100=ct.ReactorSurface(surf100, r, A=facet_fraction[2]*cat_area*m_cat)
    rsurf110=ct.ReactorSurface(surf110, r, A=facet_fraction[3]*cat_area*m_cat)

    #This is important since it adjusts the flow rates, 303 K is the temperature of the gas cylinders
    vdot = sccm  * ((ct.one_atm / gas.P) * ( 303 / 273.15)) # m^3/s 
    mass_flow_rate =  vdot* gas.density
    mflow = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)
    v = ct.PressureController(r, downstream, master=mflow, K=1e-9)

    sim=ct.ReactorNet([r])

    ##set relative and absolute tolerances on the simulation
    sim.rtol = 1.0e-12
    sim.atol = 1.0e-22
    sim.max_err_test_fails = 10000
    rxn_time=np.linspace(0,8400,8400)

    #get indices of the gas and surface species, (211) and (111) have the same order of species
    i_co2=gas.species_index('CO2')
    i_co=gas.species_index('CO')

    i_surf_Ni=surf211.species_index('site(211)')
    i_surf_CO=surf211.species_index('OC(211)')
    i_surf_O=surf211.species_index('O(211)')
    i_surf_C=surf211.species_index('C(211)')
    i_surf_CO2=surf211.species_index('CO2(211)')

    gas_mole_fracs_211=np.zeros([gas.n_species, len(rxn_time)])
    

    #get the index for each species
    idx=surf211.species_index(j)
    #get the species 
    s=surf211.species(idx)
    print(s)
    #call the original coefficients of the NASA polynomial
    original_coeffs = s.thermo.coeffs
    #create an empty array for the perturbation
    perturbed_coeffs = np.ones_like(original_coeffs)
    #fill the array initially with the original coeffs
    perturbed_coeffs[:] = original_coeffs
    #perturb the enthalpy of the adsorbate for the low and high temperature polynomial
    perturbed_coeffs[6] = original_coeffs[6] - dk
    #This is not a mistake, both values have to be perturbed by exactly the same value, otherwise discontinuity
    perturbed_coeffs[13] = original_coeffs[13] - dk
    #reset all multipliers
            
    #fit a new NASA polynomial through the data and update the thermo database
    s.thermo = ct.NasaPoly2(100.000, 5000.000, ct.one_atm, perturbed_coeffs)
    surf211.modify_species(idx, s) 

    Ti=np.zeros(len(rxn_time))
    sim.max_time_step=1e-4
    #Run the simulation
    for i in range(len(rxn_time)):
            time=rxn_time[i]
            #print(time)
            if time < 10:
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0.0, CO2:0.0'
                upstream.syncState()
            elif (time > 10) and (time < 3610):   
                sim.max_time_step=1
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:0, He:0.9013, CO2:0.0987'
                upstream.syncState()
            elif (time > 3610) and (time < 3970): 
                sim.max_time_step=10
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0, CO2:0'
                upstream.syncState()
            elif (time > 3970) and (time < 5410):
                sim.max_time_step=10
                upstream.thermo.TPX=323,p*ct.one_atm, 'Ar:1, He:0, CO2:0'   
                upstream.syncState()
                r.thermo.TP=323,ct.one_atm
                r.syncState()
            else:
                r.thermo.TP=323+beta/60.0*(rxn_time[i]-5410),ct.one_atm
                r.syncState()
             
            sim.reinitialize  
            sim.advance(time)
            Ti[i]=r.thermo.T
            gas_mole_fracs_211[:,i]=gas.X

    ind=5410
    
    max_height_CO2=np.max(gas_mole_fracs_211[i_co2,ind:])*1e6
    max_Temp_CO2=Ti[np.argmax(gas_mole_fracs_211[i_co2,ind:])+ind] 
    
    max_height_CO=np.max(gas_mole_fracs_211[i_co,ind:])*1e6
    max_Temp_CO=Ti[np.argmax(gas_mole_fracs_211[i_co,ind:])+ind]  
    print(max_height_CO2,max_Temp_CO2,max_height_CO,max_Temp_CO)
    heights_CO2_211.append(max_height_CO2)
    temps_CO2_211.append(max_Temp_CO2)

    heights_CO_211.append(max_height_CO)
    temps_CO_211.append(max_Temp_CO)
    
    #reset all multiplier
    #restore the original NASA polynomial and perturb the next species (only one at a time)
    s.thermo = ct.NasaPoly2(100.000, 5000.000, ct.one_atm, original_coeffs)
    surf211.modify_species(idx, s) 


<Species O(211)>
547.8441071296119 420.2262570940987 182.4923804621616 651.9205064094933
<Species OC(211)>
560.8637781405755 420.2262570940987 182.18792600009826 651.9205064094933
<Species C(211)>
547.844094026352 420.2262570940987 182.3048385440049 651.9205064094933
<Species CO2(211)>
560.2293638759314 420.2262570940987 182.29746148616292 651.9205064094933


In [18]:
##run the sensitivity analysis for the 110 facet

heights_CO2_110,heights_CO_110=[],[]
temps_CO2_110,temps_CO_110=[],[]
  
dH=1.0e-0
dk = dH*1000 / 8.314  # for the thermo loop, 'dk' is in fact (delta H / R)
for count,j in enumerate(surf110.species_names[1:]):    
    # import the gas model and set the initial conditions
    gas = ct.Solution(cti_file, 'gas')
    gas.TPX = T, p*ct.one_atm, 'Ar:1, He:0.0, CO2:0.0'

    #import the (111) surface
    surf111 = ct.Interface(cti_file,'surface1', [gas])
    surf111.TP = T, p*ct.one_atm
    surf111.coverages = {'site(111)':1 , 'O(111)':0.0, 'OC(111)':0.0 , 'CO2(111)':0}

    #import the (211) surface
    surf211 = ct.Interface(cti_file,'surface2', [gas])
    surf211.TP = T, p*ct.one_atm
    surf211.coverages = {'site(211)':1,  'O(211)':0.0, 'OC(211)':0.0 , 'CO2(211)':0}

    #import the (100) surface
    surf100 = ct.Interface(cti_file,'surface3', [gas])
    surf100.TP = T, p*ct.one_atm
    surf100.coverages = {'site(100)':1, 'O(100)':0.0, 'OC(100)':0.0, 'CO2(100)':0}

    #import the (110) surface
    surf110 = ct.Interface(cti_file,'surface4', [gas])
    surf110.TP = T, p*ct.one_atm
    surf110.coverages = {'site(110)':1, 'O(110)':0.0 , 'OC(110)':0.0  , 'CO2(110)':0}

    #typical Cantera setup
    upstream = ct.Reservoir(gas, name='upstream')
    downstream = ct.Reservoir(gas, name='downstream')

    r=ct.IdealGasReactor(gas, energy='off')
    r.volume=volume
    rsurf111=ct.ReactorSurface(surf111, r, A=facet_fraction[0]*cat_area*m_cat)
    rsurf211=ct.ReactorSurface(surf211, r, A=facet_fraction[1]*cat_area*m_cat)
    rsurf100=ct.ReactorSurface(surf100, r, A=facet_fraction[2]*cat_area*m_cat)
    rsurf110=ct.ReactorSurface(surf110, r, A=facet_fraction[3]*cat_area*m_cat)

    #This is important since it adjusts the flow rates, 303 K is the temperature of the gas cylinders
    vdot = sccm  * ((ct.one_atm / gas.P) * ( 303 / 273.15)) # m^3/s 
    mass_flow_rate =  vdot* gas.density
    mflow = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)
    v = ct.PressureController(r, downstream, master=mflow, K=1e-9)

    sim=ct.ReactorNet([r])

    ##set relative and absolute tolerances on the simulation
    sim.rtol = 1.0e-12
    sim.atol = 1.0e-22
    sim.max_err_test_fails = 10000
    rxn_time=np.linspace(0,8400,8400)

    #get indices of the gas and surface species, (211) and (111) have the same order of species
    i_co2=gas.species_index('CO2')
    i_co=gas.species_index('CO')

    i_surf_Ni=surf110.species_index('site(110)')
    i_surf_CO=surf110.species_index('OC(110)')
    i_surf_O=surf110.species_index('O(110)')
    i_surf_C=surf110.species_index('C(110)')
    i_surf_CO2=surf110.species_index('CO2(110)')

    gas_mole_fracs_110=np.zeros([gas.n_species, len(rxn_time)])
    

    #get the index for each species
    idx=surf110.species_index(j)
    #get the species 
    s=surf110.species(idx)
    print(s)
    #call the original coefficients of the NASA polynomial
    original_coeffs = s.thermo.coeffs
    #create an empty array for the perturbation
    perturbed_coeffs = np.ones_like(original_coeffs)
    #fill the array initially with the original coeffs
    perturbed_coeffs[:] = original_coeffs
    #perturb the enthalpy of the adsorbate for the low and high temperature polynomial
    perturbed_coeffs[6] = original_coeffs[6] - dk
    #This is not a mistake, both values have to be perturbed by exactly the same value, otherwise discontinuity
    perturbed_coeffs[13] = original_coeffs[13] - dk
    #reset all multipliers
            
    #fit a new NASA polynomial through the data and update the thermo database
    s.thermo = ct.NasaPoly2(100.000, 5000.000, ct.one_atm, perturbed_coeffs)
    surf110.modify_species(idx, s) 

    Ti=np.zeros(len(rxn_time))
    sim.max_time_step=1e-4
    #Run the simulation
    for i in range(len(rxn_time)):
            time=rxn_time[i]
            #print(time)
            if time < 10:
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0.0, CO2:0.0'
                upstream.syncState()
            elif (time > 10) and (time < 3610):   
                sim.max_time_step=1
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:0, He:0.9013, CO2:0.0987'
                upstream.syncState()
            elif (time > 3610) and (time < 3970): 
                sim.max_time_step=10
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0, CO2:0'
                upstream.syncState()
            elif (time > 3970) and (time < 5410):
                sim.max_time_step=10
                upstream.thermo.TPX=323,p*ct.one_atm, 'Ar:1, He:0, CO2:0'   
                upstream.syncState()
                r.thermo.TP=323,ct.one_atm
                r.syncState()
            else:
                r.thermo.TP=323+beta/60.0*(rxn_time[i]-5410),ct.one_atm
                r.syncState()
             
            sim.reinitialize  
            sim.advance(time)
            Ti[i]=r.thermo.T
            gas_mole_fracs_110[:,i]=gas.X

    ind=5410
    
    max_height_CO2=np.max(gas_mole_fracs_110[i_co2,ind:])*1e6
    max_Temp_CO2=Ti[np.argmax(gas_mole_fracs_110[i_co2,ind:])+ind] 
    
    max_height_CO=np.max(gas_mole_fracs_110[i_co,ind:])*1e6
    max_Temp_CO=Ti[np.argmax(gas_mole_fracs_110[i_co,ind:])+ind]  
    print(max_height_CO2,max_Temp_CO2,max_height_CO,max_Temp_CO)
    heights_CO2_110.append(max_height_CO2)
    temps_CO2_110.append(max_Temp_CO2)

    heights_CO_110.append(max_height_CO)
    temps_CO_110.append(max_Temp_CO)
    
    #reset all multiplier
    #restore the original NASA polynomial and perturb the next species (only one at a time)
    s.thermo = ct.NasaPoly2(100.000, 5000.000, ct.one_atm, original_coeffs)
    surf110.modify_species(idx, s) 


<Species O(110)>
545.8783169717364 420.2262570940987 182.3073864318985 651.9205064094933
<Species OC(110)>
550.9287729334935 420.2262570940987 179.97308202600104 652.2538794300908
<Species C(110)>
547.8440940215003 420.2262570940987 182.30483641332978 651.9205064094933
<Species CO2(110)>
547.8453148916205 420.2262570940987 182.3048390022811 651.9205064094933


In [20]:
##run the sensitivity analysis for the 110 facet

heights_CO2_100,heights_CO_100=[],[]
temps_CO2_100,temps_CO_100=[],[]
  
dH=1.0e-0
dk = dH*1000 / 8.314  # for the thermo loop, 'dk' is in fact (delta H / R)
for count,j in enumerate(surf100.species_names[1:]):    
    # import the gas model and set the initial conditions
    gas = ct.Solution(cti_file, 'gas')
    gas.TPX = T, p*ct.one_atm, 'Ar:1, He:0.0, CO2:0.0'

    #import the (111) surface
    surf111 = ct.Interface(cti_file,'surface1', [gas])
    surf111.TP = T, p*ct.one_atm
    surf111.coverages = {'site(111)':1 , 'O(111)':0.0, 'OC(111)':0.0 , 'CO2(111)':0}

    #import the (211) surface
    surf211 = ct.Interface(cti_file,'surface2', [gas])
    surf211.TP = T, p*ct.one_atm
    surf211.coverages = {'site(211)':1,  'O(211)':0.0, 'OC(211)':0.0 , 'CO2(211)':0}

    #import the (100) surface
    surf100 = ct.Interface(cti_file,'surface3', [gas])
    surf100.TP = T, p*ct.one_atm
    surf100.coverages = {'site(100)':1, 'O(100)':0.0, 'OC(100)':0.0, 'CO2(100)':0}

    #import the (110) surface
    surf110 = ct.Interface(cti_file,'surface4', [gas])
    surf110.TP = T, p*ct.one_atm
    surf110.coverages = {'site(110)':1, 'O(110)':0.0 , 'OC(110)':0.0  , 'CO2(110)':0}

    #typical Cantera setup
    upstream = ct.Reservoir(gas, name='upstream')
    downstream = ct.Reservoir(gas, name='downstream')

    r=ct.IdealGasReactor(gas, energy='off')
    r.volume=volume
    rsurf111=ct.ReactorSurface(surf111, r, A=facet_fraction[0]*cat_area*m_cat)
    rsurf211=ct.ReactorSurface(surf211, r, A=facet_fraction[1]*cat_area*m_cat)
    rsurf100=ct.ReactorSurface(surf100, r, A=facet_fraction[2]*cat_area*m_cat)
    rsurf110=ct.ReactorSurface(surf110, r, A=facet_fraction[3]*cat_area*m_cat)

    #This is important since it adjusts the flow rates, 303 K is the temperature of the gas cylinders
    vdot = sccm  * ((ct.one_atm / gas.P) * ( 303 / 273.15)) # m^3/s 
    mass_flow_rate =  vdot* gas.density
    mflow = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)
    v = ct.PressureController(r, downstream, master=mflow, K=1e-9)

    sim=ct.ReactorNet([r])

    ##set relative and absolute tolerances on the simulation
    sim.rtol = 1.0e-12
    sim.atol = 1.0e-22
    sim.max_err_test_fails = 10000
    rxn_time=np.linspace(0,8400,8400)

    #get indices of the gas and surface species, (211) and (111) have the same order of species
    i_co2=gas.species_index('CO2')
    i_co=gas.species_index('CO')

    i_surf_Ni=surf100.species_index('site(100)')
    i_surf_CO=surf100.species_index('OC(100)')
    i_surf_O=surf100.species_index('O(100)')
    i_surf_C=surf100.species_index('C(100)')
    i_surf_CO2=surf100.species_index('CO2(100)')

    gas_mole_fracs_100=np.zeros([gas.n_species, len(rxn_time)])
    

    #get the index for each species
    idx=surf100.species_index(j)
    #get the species 
    s=surf100.species(idx)
    print(s)
    #call the original coefficients of the NASA polynomial
    original_coeffs = s.thermo.coeffs
    #create an empty array for the perturbation
    perturbed_coeffs = np.ones_like(original_coeffs)
    #fill the array initially with the original coeffs
    perturbed_coeffs[:] = original_coeffs
    #perturb the enthalpy of the adsorbate for the low and high temperature polynomial
    perturbed_coeffs[6] = original_coeffs[6] - dk
    #This is not a mistake, both values have to be perturbed by exactly the same value, otherwise discontinuity
    perturbed_coeffs[13] = original_coeffs[13] - dk
    #reset all multipliers
            
    #fit a new NASA polynomial through the data and update the thermo database
    s.thermo = ct.NasaPoly2(100.000, 5000.000, ct.one_atm, perturbed_coeffs)
    surf100.modify_species(idx, s) 

    Ti=np.zeros(len(rxn_time))
    sim.max_time_step=1e-4
    #Run the simulation
    for i in range(len(rxn_time)):
            time=rxn_time[i]
            #print(time)
            if time < 10:
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0.0, CO2:0.0'
                upstream.syncState()
            elif (time > 10) and (time < 3610):   
                sim.max_time_step=1
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:0, He:0.9013, CO2:0.0987'
                upstream.syncState()
            elif (time > 3610) and (time < 3970): 
                sim.max_time_step=10
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0, CO2:0'
                upstream.syncState()
            elif (time > 3970) and (time < 5410):
                sim.max_time_step=10
                upstream.thermo.TPX=323,p*ct.one_atm, 'Ar:1, He:0, CO2:0'   
                upstream.syncState()
                r.thermo.TP=323,ct.one_atm
                r.syncState()
            else:
                r.thermo.TP=323+beta/60.0*(rxn_time[i]-5410),ct.one_atm
                r.syncState()
             
            sim.reinitialize  
            sim.advance(time)
            Ti[i]=r.thermo.T
            gas_mole_fracs_100[:,i]=gas.X

    ind=5410
    
    max_height_CO2=np.max(gas_mole_fracs_100[i_co2,ind:])*1e6
    max_Temp_CO2=Ti[np.argmax(gas_mole_fracs_100[i_co2,ind:])+ind] 
    
    max_height_CO=np.max(gas_mole_fracs_100[i_co,ind:])*1e6
    max_Temp_CO=Ti[np.argmax(gas_mole_fracs_100[i_co,ind:])+ind]  
    print(max_height_CO2,max_Temp_CO2,max_height_CO,max_Temp_CO)
    heights_CO2_100.append(max_height_CO2)
    temps_CO2_100.append(max_Temp_CO2)

    heights_CO_100.append(max_height_CO)
    temps_CO_100.append(max_Temp_CO)
    
    #reset all multiplier
    #restore the original NASA polynomial and perturb the next species (only one at a time)
    s.thermo = ct.NasaPoly2(100.000, 5000.000, ct.one_atm, original_coeffs)
    surf100.modify_species(idx, s) 


<Species O(100)>
547.8438161156647 420.2262570940987 194.75107776150242 651.5871333888956
<Species OC(100)>
547.8449293663491 420.2262570940987 181.79615665419396 652.2538794300908
<Species C(100)>
547.8440940253474 420.2262570940987 182.2477041273187 651.9205064094933
<Species CO2(100)>
547.8366871480948 420.2262570940987 182.30591353614895 651.9205064094933


In [ ]:
#Now plot the results

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import pandas as pd
%matplotlib inline

plt.rcParams['axes.titlesize']=16
plt.rcParams['axes.linewidth'] = 3 #set the value globally
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('axes', labelsize=16)
plt.rc('legend', fontsize=14)
plt.rcParams['lines.markersize'] = 10
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['xtick.direction']='in'
plt.rcParams['ytick.direction']='in'
plt.rcParams['xtick.major.size']=10
plt.rcParams['xtick.major.width']=2
plt.rcParams['ytick.major.size']=10
plt.rcParams['ytick.major.width']=2
plt.rcParams['legend.edgecolor']='k'


plt.rcParams['figure.figsize']=(14,6)
gs = gridspec.GridSpec(1, 2)
gs.update(wspace = 0.4)
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])

colormap = plt.cm.Dark2
colors = [colormap(i) for i in np.linspace(0, 1, 4)]

patch_CO = mpatches.Patch(facecolor='w', edgecolor='k',hatch='//', label='$\mathrm{CO}$')
patch_CO2 = mpatches.Patch(facecolor='k', edgecolor='k', label='$\mathrm{CO_2}$')

labels=['$\mathrm{^*O}$','$\mathrm{^*CO}$','$\mathrm{^*C}$','$\mathrm{CO_2^*}$']

#Plot the results for 111
for i in range(len(heights_CO2_111)):
    #Plot the results for 111
    ax0.barh(i,heights_CO2_111[i]-ref_height_CO2,height=0.5,color=colors[i],edgecolor='k',linewidth=2,label=labels[i])
    ax0.barh(i+0.5,heights_CO_111[i]-ref_height_CO,height=0.5,color=colors[i],hatch='//',edgecolor='k',linewidth=2)
    
    #Plot the results for 100
    ax0.barh(i+5,heights_CO2_100[i]-ref_height_CO2,height=0.5,color=colors[i],edgecolor='k',linewidth=2)
    ax0.barh(i+5.5,heights_CO_100[i]-ref_height_CO,height=0.5,color=colors[i],hatch='//',edgecolor='k',linewidth=2)
    
    #Plot the results for 110
    ax0.barh(i+10,heights_CO2_110[0]-ref_height_CO2,height=0.5,color=colors[i],edgecolor='k',linewidth=2)
    ax0.barh(i+10.5,heights_CO_110[0]-ref_height_CO,height=0.5,color=colors[i],hatch='//',edgecolor='k',linewidth=2)
    #Plot the results for 211
    ax0.barh(i+15,heights_CO2_211[0]-ref_height_CO2,height=0.5,color=colors[i],edgecolor='k',linewidth=2)
    ax0.barh(i+15.5,heights_CO_211[0]-ref_height_CO,height=0.5,color=colors[i],hatch='//',edgecolor='k',linewidth=2)

ax0.plot((0.0,0),(-0.5,19.5),linestyle='solid',color='k',linewidth=2)
ax0.plot((-5,5),(4.5,4.5),linestyle='dashed',color='k',linewidth=2)
ax0.plot((-5,5),(9.5,9.5),linestyle='dashed',color='k',linewidth=2)
ax0.plot((-5,5),(14.5,14.5),linestyle='dashed',color='k',linewidth=2)

ax0.set_ylim([-0.5,19.5])
ax0.set_xlabel('$\mathrm{\Delta c_{max}\ (ppm)}$')
ax0.set_xlim([-100,100])
ax0.bar(100,100,width=0.5,color='w',edgecolor='w',label=' ')

ax0.set_yticks([2,7,12,17])
ax0.set_yticklabels(['$\mathrm{Ni(111)}$','$\mathrm{Ni(100)}$','$\mathrm{Ni(110)}$','$\mathrm{Ni(211)}$'],rotation=90,ha='right',va='center')


handles, labels = ax0.get_legend_handles_labels()
ax0.legend(handles=[handles[0],handles[1],handles[2],handles[3],handles[4],patch_CO,patch_CO2], loc='upper right', ncol=1)

for i in range(len(heights_CO2_111)):
    #Plot the results for 111
    ax1.barh(i,temps_CO2_111[i]-ref_Temp_CO2,height=0.5,color=colors[i],edgecolor='k',linewidth=2,label=labels[i])
    ax1.barh(i+0.5,temps_CO_111[i]-ref_Temp_CO,height=0.5,color=colors[i],hatch='//',edgecolor='k',linewidth=2)

    #Plot the results for 100
    ax1.barh(i+5,temps_CO2_100[i]-ref_Temp_CO2,height=0.5,color=colors[i],edgecolor='k',linewidth=2)
    ax1.barh(i+5.5,temps_CO_100[i]-ref_Temp_CO,height=0.5,color=colors[i],hatch='//',edgecolor='k',linewidth=2)

    #Plot the results for 110
    ax1.barh(i+10,temps_CO2_110[0]-ref_Temp_CO2,height=0.5,color=colors[i],edgecolor='k',linewidth=2)
    ax1.barh(i+10.5,temps_CO_110[0]-ref_Temp_CO,height=0.5,color=colors[i],hatch='//',edgecolor='k',linewidth=2)
    #Plot the results for 211
    ax1.barh(i+15,temps_CO2_211[0]-ref_Temp_CO2,height=0.5,color=colors[i],edgecolor='k',linewidth=2)
    ax1.barh(i+15.5,temps_CO_211[0]-ref_Temp_CO,height=0.5,color=colors[i],hatch='//',edgecolor='k',linewidth=2)

ax1.plot((0.0,0),(-0.5,19.5),linestyle='solid',color='k',linewidth=2)

ax1.plot((-5,5),(4.5,4.5),linestyle='dashed',color='k',linewidth=2)
ax1.plot((-5,5),(9.5,9.5),linestyle='dashed',color='k',linewidth=2)
ax1.plot((-5,5),(14.5,14.5),linestyle='dashed',color='k',linewidth=2)

ax1.set_ylim([-0.5,19.5])
ax1.set_xlabel('$\mathrm{\Delta T_{max}\ (K)}$')
ax1.set_xlim([-5,5])
ax1.set_yticks([2,7,12,17])
ax1.set_yticklabels(['$\mathrm{Ni(111)}$','$\mathrm{Ni(100)}$','$\mathrm{Ni(110)}$','$\mathrm{Ni(211)}$'],rotation=90,ha='right',va='center')
ax1.bar(100,100,width=0.5,color='w',edgecolor='w',label=' ')


handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles=[handles[0],handles[1],handles[2],handles[3],handles[4],patch_CO,patch_CO2], loc='upper right', ncol=1)